In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
import keras
import numpy as np

from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import load_model

tf.__version__

Using TensorFlow backend.


'1.12.0'

## GPU and CPU settings
If GPU is not available, comment out the bottom block.

In [2]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
# config = tf.ConfigProto(device_count = {"GPU": 0})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.1

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

## Define input and output

In [3]:
modelname = "KnuckleFinger_LSTM_Jan_20190203_230639"
#modelname = "lstm_cnn_01_02_19"
input_path = "./ModelSnapshots/"
input_file = modelname + ".h5"

output_path = "./"
output_file = modelname + ".pb"

output_node_prefix = "output_node"

In [5]:
K.set_learning_phase(0)
net_model = load_model(input_path + input_file)

num_output = 1
pred = [None]*num_output
pred_node_names = [None]*num_output
for i in range(num_output):
    pred_node_names[i] = output_node_prefix+str(i)
    pred[i] = tf.identity(net_model.outputs[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)
output_node_prefix = pred_node_names[0]

output nodes names are:  ['output_node0']


In [6]:
net_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_38 (TimeDis (None, 50, 27, 15, 32)    320       
_________________________________________________________________
time_distributed_39 (TimeDis (None, 50, 27, 15, 32)    9248      
_________________________________________________________________
time_distributed_40 (TimeDis (None, 50, 14, 8, 32)     0         
_________________________________________________________________
time_distributed_41 (TimeDis (None, 50, 14, 8, 32)     0         
_________________________________________________________________
time_distributed_42 (TimeDis (None, 50, 3584)          0         
_________________________________________________________________
time_distributed_43 (TimeDis (None, 50, 128)           458880    
_________________________________________________________________
time_distributed_44 (TimeDis (None, 50, 64)            8256      
__________

In [7]:
sess = K.get_session()

## Export model

In [8]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)

graph_io.write_graph(constant_graph, output_path, output_file, as_text=False)

print('Saved the freezed graph at: ', (output_path + output_file))

INFO:tensorflow:Froze 18 variables.
INFO:tensorflow:Converted 18 variables to const ops.
Saved the freezed graph at:  ./KnuckleFinger_LSTM_Jan_20190203_230639.pb


## Show input and output node

In [9]:
g = tf.GraphDef()
g.ParseFromString(open(output_path + output_file, "rb").read())
s = ""
for n in g.node:
    s =s + str(n.name) + "\n"

print(s)

time_distributed_38_input
time_distributed_38/kernel
time_distributed_38/kernel/read
time_distributed_38/bias
time_distributed_38/bias/read
time_distributed_38/Reshape/shape
time_distributed_38/Reshape
time_distributed_38/convolution
time_distributed_38/BiasAdd
time_distributed_38/Relu
time_distributed_38/Reshape_1/shape
time_distributed_38/Reshape_1
time_distributed_39/kernel
time_distributed_39/kernel/read
time_distributed_39/bias
time_distributed_39/bias/read
time_distributed_39/Reshape/shape
time_distributed_39/Reshape
time_distributed_39/convolution
time_distributed_39/BiasAdd
time_distributed_39/Relu
time_distributed_39/Reshape_1/shape
time_distributed_39/Reshape_1
time_distributed_40/Reshape/shape
time_distributed_40/Reshape
time_distributed_40/MaxPool
time_distributed_40/Reshape_1/shape
time_distributed_40/Reshape_1
time_distributed_41/Reshape/shape
time_distributed_41/Reshape
time_distributed_41/Reshape_1/shape
time_distributed_41/Reshape_1
time_distributed_42/Reshape/shape
ti